In [4]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

pd.options.display.max_columns = None

In [5]:
datos = ['https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=60',
        'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=120', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=180',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=240', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=300',
         'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=360', 'https://sofifa.com/players?type=all&aeh=21&oah=70&ptl=80&offset=420']

In [6]:
nombre = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    productos = soup.find_all("td", {"class": "col-name"})
    
    jugadores = {'jugador': []}
    
    for x in productos:
        jugadores['jugador'].append(x.find('a').get('href')) 
    
    jugadores_top = pd.DataFrame(jugadores)
    
    jugadores_top['jug'] = jugadores_top['jugador'].str.extract(r'(?<=\/player\/\d\d\d\d\d\d\/)(.*)(?=\/220057)')
    
    jugadores_fichar = jugadores_top.dropna()
    jugadores_fichar = jugadores_fichar.reset_index(drop=True)
    
    equipos = jugadores_fichar.drop(['jugador'], axis=1)
    
    nombre = pd.concat([nombre, equipos], axis=0)

In [7]:
nombre = nombre.reset_index(drop=True)
nombre.head(2)

,jug
0,gaston-alvarez
1,abdessamad-ezzalzouli


In [8]:
equipo = pd.DataFrame()

for i in datos:
    url = i
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    html.status_code
    
    productos = soup.find_all("td", {"class": "col-name"})
    
    jugadores = {'jugador': []}
    
    for x in productos:
        jugadores['jugador'].append(x.find('a').get('href')) 
    
    jugadores_top = pd.DataFrame(jugadores)
    
    jugadores_top['team1'] = jugadores_top['jugador'].str.extract(r'(?<=\/team\/)(.*)(?=\/)')
    jugadores_top['team'] = jugadores_top['team1'].str.extract(r'(?<=\d\/)(.*)')
    
    equipos = jugadores_top.drop(['jugador','team1'], axis=1)
    
    equip = equipos.dropna()
    equip = equip.reset_index(drop=True)
    
    equipo = pd.concat([equipo, equip], axis=0)

In [9]:
equipo = equipo.reset_index(drop=True)
equipo.head(2)

,team
0,getafe-cf
1,fc-barcelona


In [10]:
nomyequip = pd.concat([nombre, equipo], axis=1)
nomyequip.head(2)

,jug,team
0,gaston-alvarez,getafe-cf
1,abdessamad-ezzalzouli,fc-barcelona


In [13]:
nomyequip = nomyequip.drop(nomyequip.index[[31,80,89,95,116,158,208,263]])
nomyequip = nomyequip.reset_index(drop=True)

In [14]:
nuevas_columnas = {
    'jug' : 'Nombre jugador', 
    'team': "Equipo actual"}

In [15]:
nomyequip.rename(columns = nuevas_columnas, inplace = True)
nomyequip.head(2)

,Nombre jugador,Equipo actual
0,gaston-alvarez,getafe-cf
1,abdessamad-ezzalzouli,fc-barcelona


In [16]:
nomyequip.to_csv('../../data/precio/nombres_equipos.csv')